First you need to run, convert to pytorch, and download unsing this colab notebook https://colab.research.google.com/drive/1ewICGkA-rxla33E5_gYCSTnYSW1_actK#scrollTo=Pdoa7YYT7R1s

This is because I can't get tensorflow working on my local pc, plus training works far better on a TPU

## Run GPT-2

- from https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/run_gpt2.py

In [1]:
#!/usr/bin/env python3

import argparse
import logging
from tqdm import trange

import torch
import torch.nn.functional as F
import numpy as np

from pytorch_pretrained_bert import GPT2LMHeadModel, GPT2Tokenizer

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

def top_k_logits(logits, k):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        return torch.where(logits < batch_mins, torch.ones_like(logits) * -1e10, logits)

def sample_sequence(model, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0, device='cuda', sample=True):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
        context = torch.tensor(context, device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = torch.full((batch_size, 1), start_token, device=device, dtype=torch.long)
    prev = context
    output = context
    past = None
    with torch.no_grad():
        for i in trange(length):
            logits, past = model(prev, past=past)
            logits = logits[:, -1, :] / temperature
            logits = top_k_logits(logits, k=top_k)
            log_probs = F.softmax(logits, dim=-1)
            if sample:
                prev = torch.multinomial(log_probs, num_samples=1)
            else:
                _, prev = torch.topk(log_probs, k=1, dim=-1)
            output = torch.cat((output, prev), dim=1)
    return output




In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name_or_path', type=str, default='gpt2', help='pretrained model name or path to local checkpoint')
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--nsamples", type=int, default=1)
parser.add_argument("--batch_size", type=int, default=-1)
parser.add_argument("--length", type=int, default=-1)
parser.add_argument("--temperature", type=float, default=1.0)
parser.add_argument("--top_k", type=int, default=0)
parser.add_argument("--fp16", type=str, default='')
parser.add_argument('--unconditional', action='store_true', help='If true, unconditional generation.')


argv = """ \
--model_name_or_path /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch \
--fp16 O2 \
--length 128 \
""".replace('\n', '')
args = parser.parse_args(argv.split())
print(args)


Namespace(batch_size=-1, fp16='O2', length=128, model_name_or_path='/home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch', nsamples=1, seed=0, temperature=1.0, top_k=0, unconditional=False)


In [3]:

if args.batch_size == -1:
    args.batch_size = 1
assert args.nsamples % args.batch_size == 0

np.random.seed(args.seed)
torch.random.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

enc = GPT2Tokenizer.from_pretrained(args.model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(args.model_name_or_path)
model.to(device)
model.eval()

if args.length == -1:
    args.length = model.config.n_ctx // 2
elif args.length > model.config.n_ctx:
    raise ValueError("Can't get samples longer than window size: %s" % model.config.n_ctx)

07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading special tokens file /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch/special_tokens.txt
07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading vocabulary file /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch/vocab.json
07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading merges file /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch/merges.txt
07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   loading weights file /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch/pytorch_model.bin
07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   loading configuration file /home/wassname/Downloads/checkpoint_reddit_ml_gpt2med_pytorch/run1_pytorch/config.json
07/15/2019 16:20:21 - INFO - pytorch_pretrained_bert.modeling_gpt2 

In [4]:
# # Prepare model for FP16 and distributed training if needed (order is important, distributed should be the last)
# # optimizer = OpenAIAdam(model.parameters(), lr=args.lr)
# if args.fp16:
#     from apex import amp  # Apex is only required if we use fp16 training
# model, optimizer = amp.initialize(model, opt_level=args.fp16)

    ****S
    https://blog.photoeditorsdk.com/from-2d-to-3d-photo-editing-948690b7b45e
    [P] Using deep learning to go from 2D to 3D photo editing

    ****ES 8tykor

    ****T 8tykor
    &gt; we were wondering if we could bring this magic to any type of portrait image. 

    I'm sorry, but I can't take any article that uses the word 'magic' seriously .. becuase we know it ain't magic.
    ****ET e1dd3zm

    ****S
    https://pjreddie.com/media/files/papers/YOLOv3.pdf
    [R] YOLOv3: An Incremental Improvement

    ****ES 877ahu

    ****T 877ahu
    hail to the meme lord
    ****ET dwcmpf1

In [5]:
def format_submission(thing):
    return f'''****S
{thing.get('url','')}
{thing['title']}
{thing.get('selftext', '')}
****ES {thing['id']}

'''

def format_top_level_comment(thing):
    return f'''****T {thing['parent_id'][3:]}
{thing['body']}
****ET {thing['id']}

'''

def format_reply(thing):
    return f'''****R {thing['parent_id'][3:]}
{thing['body']}
****ER {thing['id']}

'''



In [6]:
# input_text = chat2input("I don't think so", ['Scientists have found a new way to deep learn: Wide learning', 'This makes no sense, do they mean adding larger linear layers?'])
# print(input_text)

In [12]:
from IPython.display import HTML, display
import re
import html

def split_output(output_text):
    # Remove open tags
    s = re.sub('\*\*\*\*[^E]( \w+)?\n', '', output_text)
    # Split by closing tag and id
    ss = re.split('\*\*\*\*E[A-Z] \w+\n?', s)
    # Strip and remove empties
    ss = [s.strip() for s in ss if s]
    return ss
def outputformat(output_text):
    ss = split_output(output_text)
    ss = ('\n'+'-'*80+'\n').join(ss)
    return html.unescape(ss)
    
    
output_text='''****S
https://www.reddit.com/r/shittyaskscience/comments/98o4i4/making_tacos_i_ran_out_of_cilantro_what_brand_of/
Making tacos, I ran out of cilantro. What brand of dishsoap should I use as a substitute, and how much of it?

****ES 98o4i4

****T 98o4i4
Fuck Cilantro and it's Soapy "freshness".
****ET e4iyj9x

****T 98o4i4
My girlfriend thinks cilantro tastes like dish soap. I fucking love cilantro but hate dish soap. Also, I hate cucumbers and watermelons which she loves although neither taste like dish soap to either of us. These are all linked to specific genes we carry. 
****ET e4isg2p

****T 98o4i4
This is the best shittyaskscience I’ve seen because no one know what you’re talking about so far and I’m one of the small percentage who tastes the fucking dishy soapy water. Cheers.
****ET e4ipex6

****R e4ipex6
To be fair, you have to have a very high IQ to understand the nuance of cilantro flavor. ;)
****ER e4jo1bu
'''
print(outputformat(output_text))

https://www.reddit.com/r/shittyaskscience/comments/98o4i4/making_tacos_i_ran_out_of_cilantro_what_brand_of/
Making tacos, I ran out of cilantro. What brand of dishsoap should I use as a substitute, and how much of it?
--------------------------------------------------------------------------------
Fuck Cilantro and it's Soapy "freshness".
--------------------------------------------------------------------------------
My girlfriend thinks cilantro tastes like dish soap. I fucking love cilantro but hate dish soap. Also, I hate cucumbers and watermelons which she loves although neither taste like dish soap to either of us. These are all linked to specific genes we carry.
--------------------------------------------------------------------------------
This is the best shittyaskscience I’ve seen because no one know what you’re talking about so far and I’m one of the small percentage who tastes the fucking dishy soapy water. Cheers.
----------------------------------------------------------

In [8]:
import uuid
def chat2input(history):
    """Transform a chat history and reply into a format the model expects."""
    last_id = uuid.uuid4().hex[:7]
    formatted = []
    if history:
        f = format_submission(dict(id=last_id, url='', title=history[0]))
        formatted.append(f)
    if len(history)>1:
        this_id = uuid.uuid4().hex[:7]    
        f = format_top_level_comment(dict(body=history[1], id=this_id, parent_id='t3_'+last_id))
        formatted.append(f)
        last_id = this_id
    for comment in history[2:]:
        this_id = uuid.uuid4().hex[:7]    
        f = format_reply(dict(body=comment, id=this_id, parent_id='t3_'+last_id))
        formatted.append(f)
        last_id = this_id 
    return ''.join(formatted)

In [25]:
class Chat(object):
    def __init__(self, args, model, enc):
        self.args = args
        self.enc = enc
        self.model = model
        self.history = []
        
    def sample(self,history):
        args = self.args
        raw_text = chat2input(history)
        
        logger.debug('raw_text: %s', raw_text)
        context_tokens = enc.encode(raw_text)
        generated = 0
        out = sample_sequence(
            model=self.model, length=args.length,
            context=context_tokens,
            start_token=None,
            batch_size=args.batch_size,
            temperature=args.temperature, top_k=args.top_k, device=device
        )
        out = out[:, len(context_tokens):].tolist()
        
        text = ''
        for i in range(args.batch_size):
            generated += 1
            s = self.enc.decode(out[i])
            text += s
        # Just take one comment
        text2 = split_output(text)[0] 
#         assert '***' not in text2
        return text2
    
    def all_text(self):
        return chat2input(self.history)
    
    def reply(self, prompt):
        self.history.append(prompt)  
        reply = self.sample(self.history)   
        self.history.append(reply)
        return reply

In [26]:
chat = Chat(args, model, enc)
text_raw = chat.reply("Scientists have found a new way to deep learn")
text_raw = chat.reply('This makes no sense, do they mean adding larger linear layers?')
text_raw = chat.reply('Did you read the article?')
text_raw = chat.reply('I think I love you. Do you love me? In my dream you are my wife and I am also your wife.')
text_raw = chat.reply('Fuck you buddy')
text = outputformat(text_raw)
print(outputformat(chat.all_text()))

100%|██████████| 128/128 [00:06<00:00, 21.30it/s]

****S

Scientists have found a new way to deep learn

****ES 230a99f

****T 230a99f
It's my more scifi ending I’ve been tinkering with. Deep learning sounds like mashing an egg with lots of gasoline.
****ET aef9137

****R aef9137
This makes no sense, do they mean adding larger linear layers?
****ER d3f8152

****R d3f8152
Okay, so this is what I've been trying to do and I’ve looked at the conventional short code methods for what’s best, but nocorrecting algorithms seemed good enough for me.
****ER 6f5b5cf

****R 6f5b5cf
Did you read the article?
****ER d171573

****R d171573
hmm weird, am I supposed to have been there? But sounds like a work of high literature!
****ER 729439b

****R 729439b
I think I love you. Do you love me? In my dream you are my wife and I am also your wife.
****ER 44d0fb0

****R 44d0fb0
What’s the scientific term for someone with autism?
****ER 2f38f4b

****R 2f38f4b
Fuck you buddy
****ER f299fc3

****R f299fc3
CFD is better than parallelism, under triple threat of 

In [27]:
print(outputformat(chat.all_text()))

Scientists have found a new way to deep learn
--------------------------------------------------------------------------------
It's my more scifi ending I’ve been tinkering with. Deep learning sounds like mashing an egg with lots of gasoline.
--------------------------------------------------------------------------------
This makes no sense, do they mean adding larger linear layers?
--------------------------------------------------------------------------------
Okay, so this is what I've been trying to do and I’ve looked at the conventional short code methods for what’s best, but nocorrecting algorithms seemed good enough for me.
--------------------------------------------------------------------------------
Did you read the article?
--------------------------------------------------------------------------------
hmm weird, am I supposed to have been there? But sounds like a work of high literature!
--------------------------------------------------------------------------------
I t

In [24]:
chat = Chat(args, model, enc)
while True:
    # Get prompt
    raw_text = input("Model prompt >>> ")
    while not raw_text:
        print('Prompt should not be empty!')
        raw_text = input("Model prompt >>> ")
        
    # Reply
    output = chat.reply(raw_text)
    print(output)

Model prompt >>> Hi buddy


100%|██████████| 128/128 [00:06<00:00, 20.74it/s]


They got last year, it was magician school. Have you heard of the swaggera?
Model prompt >>> Oh interesting, no I haven't. Is that a dance?


100%|██████████| 128/128 [00:06<00:00, 20.69it/s]


Swaggera? wtf is this even a thing?  OK I'm blaming it on people using giant B-5 cubes with light bulbs, cheers!
Model prompt >>> Well you brought it up. Or was it the other guy?


...

vote from "yes" boxes, lock them in the slack and watch each other complain about illicit nukes and piracy.
Model prompt >>> Well you lost me there


100%|██████████| 128/128 [00:06<00:00, 20.79it/s]


If they deliver then this is a thing on Facebook
Model prompt >>> Yup I'm lost


100%|██████████| 128/128 [00:06<00:00, 20.90it/s]


AssertionError: 

In [22]:
output

"Me neither nor can't _"

In [23]:
raw_text

'Hi buddy'